In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("abalone.csv")

In [3]:
# 특징과 라벨 분리
X = df.drop(['Age'], axis = 1)
Y = df['Age']

In [4]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
Train_X.corr() # 특징 간 상관 행렬 출력 => 얼핏봐도 특징 간 선형 관계가 존재

,Length,Diameter,Height,Wholeweight,Shuckedweight,Visceraweight,Shellweight
Length,1.000000,0.986331,0.809434,0.925405,0.898610,0.903386,0.895786
Diameter,0.986331,1.000000,0.815719,0.925601,0.893996,0.899829,0.904050
Height,0.809434,0.815719,1.000000,0.799875,0.755448,0.778875,0.798438
Wholeweight,0.925405,0.925601,0.799875,1.000000,0.968246,0.966389,0.953806
Shuckedweight,0.898610,0.893996,0.755448,0.968246,1.000000,0.932727,0.880176
Visceraweight,0.903386,0.899829,0.778875,0.966389,0.932727,1.000000,0.905409
Shellweight,0.895786,0.904050,0.798438,0.953806,0.880176,0.905409,1.000000


In [6]:
# VIF 기준 특징 선택, VIF 계산
from sklearn.linear_model import LinearRegression as LR
VIF_dict = dict() # 빈 딕셔너리 생성
for col in Train_X.columns:
    # 1개의 컬럼을 기준으로 다른 라벨과의 비교로 모델 생성
    model = LR().fit(Train_X.drop([col], axis = 1), Train_X[col])
    # LinearRegression의 score가 r2 점수임
    r2 = model.score(Train_X.drop([col], axis = 1), Train_X[col]) 
    VIF = 1 / (1 - r2)
    VIF_dict[col] = VIF # 딕셔너리에 결과 추가

In [7]:
# Height를 제외하곤 VIF가 모두 높다
# 이러한 상황에서는 사실 PCA를 사용하는 것이 좋다.
VIF_dict 

{'Length': 39.43849833918085,
 'Diameter': 40.492849239834015,
 'Height': 3.204948043021875,
 'Wholeweight': 97.86250558936253,
 'Shuckedweight': 25.697425536549265,
 'Visceraweight': 17.0028393648043,
 'Shellweight': 19.491294050676053}

In [8]:
# mean_absolute_error는 실제 값과 예측 값의 차이(Error)를 절대값으로 변환해 평균화 한다
# 에러에 절대값을 취하기 때문에 에러의 크기 그대로 반영된다
# 예측 결과물의 에러가 10이 나온 것이 5로 나온 것보다 2배가 나쁜 도메인에서 쓰기 적합한 산식이다
# 에러에 따른 손실이 선형적으로 올라가거나 이상치가 많을 때 적합하다. 

from sklearn.neural_network import MLPRegressor as MLP
from sklearn.metrics import mean_absolute_error as MAE

In [9]:
# 전체 특징을 모두 사용하였을 때
model = MLP(random_state = 2313, max_iter = 500)
model.fit(Train_X, Train_Y)
pred_Y = model.predict(Test_X)
score = MAE(Test_Y, pred_Y)
print(score)

1.6052171190360358


In [10]:
# VIF 점수가 30점 미만인 특징만 사용하였을 때 
selected_features = [key for key, val in VIF_dict.items() if val < 30] 

model = MLP(random_state = 2313, max_iter = 500)
model.fit(Train_X[selected_features], Train_Y)
pred_Y = model.predict(Test_X[selected_features])
score = MAE(Test_Y, pred_Y)
print(score)
# 전체특징을 모두 이용했을 때 보다 성능이 약간 좋아졌다.

1.6047893121350008


c:\users\jaehwan\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [11]:
# PCA 사용, 차원축소
from sklearn.decomposition import PCA
# 3개의 중성분으로 데이터를 변환
PCA_model = PCA(n_components = 3).fit(Train_X)

Train_Z = PCA_model.transform(Train_X)
Test_Z = PCA_model.transform(Test_X)

print(Train_Z.shape)
# 차원의 갯수가 3개로 줄어들었다

(3132, 3)


In [12]:
model = MLP(random_state = 2313, max_iter = 500)
model.fit(Train_Z, Train_Y)
pred_Y = model.predict(Test_Z)
score = MAE(Test_Y, pred_Y)
print(score)
# 이전보다 성능이 더 좋아졌다.

1.5383788443381812
